In [56]:
import os
import re
import nltk
from nltk import word_tokenize , sent_tokenize , pos_tag
from tdmh import *
from os.path import join
import stanza

In [4]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('wordnet')
nltk.download('sentiwordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\RS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\RS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\RS\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\RS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\RS\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [30]:
texts = []
dir = r'C:\Users\RS\Desktop\FP_T&DM\Corpus'

for entry in os.listdir(dir):
    # dir + subdirectory
    path = os.path.join(dir,entry)
    if os.path.isdir(path):
        
        for file in os.listdir(path):
            # dir + subdirectory + file 
            file_path = join(path,file)
            texts.append( file_path )
            print( file_path )

In [11]:
stanza.download('pt')       
nlp = stanza.Pipeline('pt')

def get_title(path):
    title = os.path.basename(path)
    if re.search( r'txt$' , title ):
        # Remove txt extension
        title = title[ :title.index('.txt') ]
        # remove commas and dots
        title = re.sub( r'[.,]' , '' , title )
    return title

2022-04-06 00:09:26 INFO: Downloading default packages for language: pt (Portuguese)...


2022-04-06 00:10:58 INFO: Finished downloading models and saved to C:\Users\RS\stanza_resources.
2022-04-06 00:10:58 INFO: Loading these models for language: pt (Portuguese):
| Processor | Package |
-----------------------
| tokenize  | bosque  |
| mwt       | bosque  |
| pos       | bosque  |
| lemma     | bosque  |
| depparse  | bosque  |

2022-04-06 00:10:58 INFO: Use device: cpu
2022-04-06 00:10:58 INFO: Loading: tokenize
2022-04-06 00:10:59 INFO: Loading: mwt
2022-04-06 00:10:59 INFO: Loading: pos
2022-04-06 00:10:59 INFO: Loading: lemma
2022-04-06 00:10:59 INFO: Loading: depparse
2022-04-06 00:11:00 INFO: Done loading processors!


In [14]:
with open( 'Corpus\FP_Autopsicografia.txt' , encoding = 'utf-8') as file:
    full_text = file.read() 
    
print(full_text)
    

O poeta é um fingidor
Finge tão completamente
Que chega a fingir que é dor
A dor que deveras sente.

E os que lêem o que escreve,
Na dor lida sentem bem,
Não as duas que ele teve,
Mas só a que eles não têm.

E assim nas calhas de roda
Gira, a entreter a razão,
Esse comboio de corda
Que se chama coração.



In [38]:
import tdmh
import re
from nltk.tokenize import word_tokenize

def remove_punctuation(words):
    new_list= []
    for w in words:
        if w.isalnum():
            new_list.append( w )
    return new_list
    
with open( 'Corpus\FP_Autopsicografia.txt' , encoding = 'utf-8') as poem:
    full_text = poem.read()

words = word_tokenize(full_text)
words = remove_punctuation(words)

print(f'These are the words in this poem: {words}.')
print(f'There are { len(words) } tokens in this poem.')
print(f'There is {len(full_text)} characters in this poem.')

These are the words in this poem: ['O', 'poeta', 'é', 'um', 'fingidor', 'Finge', 'tão', 'completamente', 'Que', 'chega', 'a', 'fingir', 'que', 'é', 'dor', 'A', 'dor', 'que', 'deveras', 'sente', 'E', 'os', 'que', 'lêem', 'o', 'que', 'escreve', 'Na', 'dor', 'lida', 'sentem', 'bem', 'Não', 'as', 'duas', 'que', 'ele', 'teve', 'Mas', 'só', 'a', 'que', 'eles', 'não', 'têm', 'E', 'assim', 'nas', 'calhas', 'de', 'roda', 'Gira', 'a', 'entreter', 'a', 'razão', 'Esse', 'comboio', 'de', 'corda', 'Que', 'se', 'chama', 'coração'].
There are 64 words in this poem.
There is 305 characters in this poem.
